<a href="https://colab.research.google.com/github/jsansao/teic-20231/blob/main/TEIC_Licao25_LSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificação de Texto com LSA (Latent Semantic Analysis)

Este notebook demonstra como usar a **Análise Semântica Latente (LSA)** para resolver o mesmo problema de classificação de texto (computação gráfica vs. criptografia).

LSA não é um classificador. É uma técnica de **redução de dimensionalidade** e **extração de características**.

O fluxo de trabalho será:
1.  **Texto Bruto** -> Aplicar `TfidfVectorizer`
2.  **Matriz TF-IDF** (Alta dimensão, Esparsa) -> Aplicar `TruncatedSVD` (o motor do LSA)
3.  **Matriz LSA** (Baixa dimensão, Densa) -> Alimentar um `LogisticRegression`
4.  Avaliar o classificador.

In [1]:
# Célula 1: Importações

import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD # Esta é a implementação do LSA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline # Usaremos um Pipeline para simplificar!

## Passo 1: Carregar os Dados

Carregamos os mesmos dados de treino e teste.

In [2]:
# Célula 2: Carregar os dados
categorias = ['comp.graphics', 'sci.crypt']
dados = fetch_20newsgroups(subset='all', categories=categorias, shuffle=True, random_state=42, remove=('headers', 'footers', 'quotes'))
X_train_text, X_test_text, y_train, y_test = train_test_split(dados.data, dados.target, test_size=0.2, random_state=42)

print(f"Amostras de treino: {len(X_train_text)}")
print(f"Amostras de teste: {len(X_test_text)}")

Amostras de treino: 1571
Amostras de teste: 393


## Passo 2: Criar o Pipeline (TF-IDF -> LSA -> Classificador)

Usar um `Pipeline` do Scikit-learn é a forma mais limpa e correta de encadear estas três etapas. O Pipeline garante que o `TfidfVectorizer` e o `TruncatedSVD` sejam ajustados (fitted) *apenas* nos dados de treino, e depois usados para transformar (transform) os dados de teste, evitando vazamento de dados (data leakage).

In [3]:
# Célula 3: Definindo os componentes do Pipeline

# 1. O Vetorizador TF-IDF
# Usaremos 5000 features, como nos exemplos anteriores
tfidf = TfidfVectorizer(max_features=5000, stop_words='english')

# 2. O LSA (TruncatedSVD)
# Vamos reduzir as 5000 dimensões do TF-IDF para 100 "tópicos" latentes.
N_COMPONENTES_LSA = 100
lsa = TruncatedSVD(n_components=N_COMPONENTES_LSA, random_state=42)

# 3. O Classificador
# Regressão Logística funciona muito bem com a saída densa do LSA
clf = LogisticRegression(random_state=42)

# Criar o Pipeline que executa os passos em sequência
pipeline_lsa = Pipeline([
    ('tfidf', tfidf), # Passo 1: Calcula TF-IDF
    ('lsa', lsa),   # Passo 2: Reduz dimensionalidade com LSA
    ('clf', clf)    # Passo 3: Classifica usando as features do LSA
])

## Passo 3: Treinar e Avaliar o Modelo

In [4]:
# Célula 4: Treinar o Pipeline
print("Treinando o pipeline (TF-IDF -> LSA -> Regressão Logística)...")

# O .fit() treina todos os passos em sequência nos dados de treino
pipeline_lsa.fit(X_train_text, y_train)

print("Treinamento concluído.")

Treinando o pipeline (TF-IDF -> LSA -> Regressão Logística)...
Treinamento concluído.


In [5]:
# Célula 5: Avaliar o Pipeline

# O .predict() aplica o TF-IDF, o LSA e o Classificador nos dados de teste
y_pred = pipeline_lsa.predict(X_test_text)

acuracia_lsa = accuracy_score(y_test, y_pred)

print(f"Acurácia (TF-IDF + LSA(100) + LogReg): {acuracia_lsa:.4f}")

Acurácia (TF-IDF + LSA(100) + LogReg): 0.9059


## (Opcional) O que o LSA aprendeu?

Podemos inspecionar os "tópicos" (componentes) que o LSA aprendeu, olhando quais palavras do TF-IDF têm mais peso em cada tópico.

In [6]:
# Célula 6: Inspecionando os tópicos do LSA

# Pegar os nomes das features (palavras) do passo 'tfidf'
termos_tfidf = pipeline_lsa.named_steps['tfidf'].get_feature_names_out()

# Pegar os componentes (tópicos) do passo 'lsa'
componentes_lsa = pipeline_lsa.named_steps['lsa'].components_

print(f"Formato dos componentes do LSA (tópicos, features): {componentes_lsa.shape}\n")

def exibir_topicos(model_components, feature_names, n_top_words=10):
    print("\n--- Tópicos Latentes encontrados pelo LSA ---")
    for i, topico in enumerate(model_components[:5]): # Exibir os 5 primeiros tópicos
        # Pega os índices das palavras com maior peso (positivo e negativo)
        indices_top_positivos = topico.argsort()[::-1][:n_top_words]
        palavras_top_positivas = [feature_names[i] for i in indices_top_positivos]

        print(f"Tópico {i}: {', '.join(palavras_top_positivas)}")

exibir_topicos(componentes_lsa, termos_tfidf)

Formato dos componentes do LSA (tópicos, features): (100, 5000)


--- Tópicos Latentes encontrados pelo LSA ---
Tópico 0: key, chip, encryption, clipper, government, use, know, like, don, people
Tópico 1: key, chip, clipper, government, encryption, keys, escrow, law, nsa, phone
Tópico 2: key, bit, s1, keys, serial, bits, number, s2, 80, block
Tópico 3: image, file, jpeg, files, format, gif, government, images, 24, color
Tópico 4: thanks, does, know, advance, anybody, format, looking, files, hi, windows


## Conclusão

Como você pode ver nos resultados da Célula 6, o LSA consegue agrupar palavras semanticamente relacionadas. Por exemplo, um tópico provavelmente conterá ("key", "encryption", "clipper", "chip", "security") e outro conterá ("graphics", "image", "3d", "polygon", "computer").

O classificador de Regressão Logística então aprende a tomar decisões com base nesses "conceitos" (Tópico 0, Tópico 1, etc.) em vez de em palavras individuais.

Essa abordagem (TF-IDF + LSA) foi o estado da arte por muitos anos antes da ascensão das Word Embeddings (Word2Vec, GloVe, FastText) e dos Transformers (BERT).